In [1]:
family_group = "6.Leptidea"


def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [2]:
import os
from Bio import SeqIO
import subprocess



output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Vanessa_tameamea"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            error_exons += coordinate_file_list[i]
        
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == "24":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()

            if "*" in translated[:-1]:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Leptidea_juvernica
GCA_949711755.1_LjuvernicaM_genomic.fna
23465986 23466071
Getting Gene


/home/saurav/.local/lib/python3.8/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


MDTLDDSEKNAKISDSAYSNSCTDSQAR
>Leptidea_juvernica_Exon_1_CATIXQ010000001.1_23465986_23466071_left_0_right_2
ATGGATACTCTAGATGATTCAGAAAAGAATGCAAAAATATCGGACTCGGCGTATTCAAACAGTTGTACCGATAGTCAAGCAAGGag
23463649 23463732
Getting Gene
>Leptidea_juvernica_Exon_2_CATIXQ010000001.1_23463649_23463732_left_1_right_2
taataGTTCACATTCTACACATTGTGGCAGTCACTCAAGTGGCAGCAGTGGATATGGAGGAGAACCTACCAAATCCGGTTATAG
23461671 23461822
Getting Gene
>Leptidea_juvernica_Exon_3_CATIXQ010000001.1_23461671_23461822_left_1_right_1
TACTAATAATCTCAGCCAACCCCCGGAAAAGAAAAGGAAGATAAACAAGAAGAAGACAGCAGAATGtgaaaaacttaaaagtaaaaagaagGACCCAAGTCCTCCAAAATTTGAAATAGAAGTAACATTGAAAGAACAAGAAGCGAATGGCG
23460184 23460296
Getting Gene
>Leptidea_juvernica_Exon_4_CATIXQ010000001.1_23460184_23460296_left_2_right_0
CAATTGAAGTTTCGAAGACACAAATGTCGGAATTCGGAAGTAACGTTGAAAATGTGGACAGCTCAACGGAACCAGAACCACCGAATTGCGCAAACgctattaatgataataag
23459781 23459837
Getting Gene
>Leptidea_juvernica_Exon_5_CATIXQ010000001.1_23459781_23459837_left_0_right_0
AAAAGTTCCAGCAATATT

>Leptidea_reali_Exon_11_CATIXK010000002.1_20855719_20855838_left_0_right_0
GGTCCAACTAACCCAGATGTGTTTCAAAGCCCGAACCCggaagtacaaaataaatttcctGAGGATGATGATAATAAGGCCAAATCGTTGAGGGAAAACATCATAAAGATACTAAATGAG
20855164 20855313
Getting Gene
>Leptidea_reali_Exon_12_CATIXK010000002.1_20855164_20855313_left_0_right_0
cCTACAACTAAATCCGAAGATTTAGCCAAACAGGAGATAGCGATTCGATGCCAGAAAGTAGCCTCTATCATTGAAAGCTTAAACGAAAAAGCACCGAATGTAGACAAAGAGATTTGGATCGATATTCAAGAGCCTGAAAATGTtaattat
20853999 20854117
Getting Gene
>Leptidea_reali_Exon_13_CATIXK010000002.1_20853999_20854117_left_0_right_2
GACAGGGATTCAGTTCTGCTTGGGAGCATATCTCCTCACACCGACAGTATTGTCAGCAAGGCGAGCACCGAAACTCAAGTCAACTACAATCAATTGAACACCGATGAAAATTTAGACAG
20850912 20851013
Getting Gene
>Leptidea_reali_Exon_14_CATIXK010000002.1_20850912_20851013_left_1_right_2
GTACTTCAACCGTTGGTTAGAACATTGTGTGGAAGAGGAGATAGCGGAGGCCGAACCTAATAATGGTGAAGCTAATTTGGAGAATTCTAATGAGACTAATGg
20847444 20847581
Getting Gene
>Leptidea_reali_Exon_15_CATIXK010000002.1_20847444_20847581_left_1_right_2
GGAATT

>Leptidea_sinapis_Exon_21_NC_066312.1_12101422_12101583_left_0_right_0
AGTACAAACGCAGTTACTTTAAGCCAGAACAACGAGAAAGATCCGATTGCCAACAGCGGGACGAATGCAGATAAGACTGATGATGAATCTTGGTTTTCTTCATTTTCCTCTCACATATATAAGAGTGAAACTGGCAGCCCAGAGGAATCCGAAATGAAAAAG
12103768 12103854
Getting Gene
>Leptidea_sinapis_Exon_22_NC_066312.1_12103768_12103854_left_0_right_0
AAATCTACGATATCATCACTACAAACCAATCGTCAAACAATAAATAAGGTAGCAATGAAGAAAATAGATCCCCCGTGGCTGGATGAT
12105113 12105214
Getting Gene
>Leptidea_sinapis_Exon_23_NC_066312.1_12105113_12105214_left_0_right_0
GTGTGCCTTACTCCTgagataatatacaaatatcaaGTCAAAACCAGAAGCTTGGCGGAAGTACTTGCGGAAGACAAGAAGAAACTCGAAGCCCTAGAGCAG
12106588 12106722
Getting Gene
>Leptidea_sinapis_Exon_24_NC_066312.1_12106588_12106722_left_0_right_0
ccGAGCTTGGTCAACGAACAATTGAGCCAGTTGTATTACGAGCTCAAATTAGAGGGACTTCCGGAAAAATTGACTTTAGAGGAAGGAATAACGAGTTCTAGCAGTTCTGGTGATGATACCACTAGTAACACAGAG
12109248 12109352
Getting Gene
>Leptidea_sinapis_Exon_25_NC_066312.1_12109248_12109352_left_0_right_0
GATCGAAGAAGGAAACGTGAACATGGCAAAC